In [1]:
# INITIALIZATION

import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator
from qiskit import *
from qiskit.circuit.library import ZZFeatureMap
from qiskit.compiler import transpile, assemble
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit_machine_learning.algorithms import QSVR
from qiskit_machine_learning.kernels import QuantumKernel
from ibm_quantum_widgets import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.linalg import expm
import math
import numpy as np
import matplotlib.pyplot as plt
import random
import time

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

print("Done!")

Done!


In [ ]:
# OBTAIN DATA
import pandas as pd

!pip install git+https://github.com/jundongl/scikit-feature.git

#Leer fichero de datos
data = pd.read_csv('ETL_Jerarquia_0_v1.csv', sep=';') #leer csv
#Guardar nombres de las columnas
colNames = data.columns
#Crear una copia del dataframe original
OriginalData=data.copy()

#Preprocesar columnas 0-1 (IPs) y 5-6 (valores con millares representados con puntos) 
#Quitar puntos de las columnas
#Columna de IPs
print(data.iloc[:,0].head(n=4))
for i in np.array([0,1,5,6]):
    data.iloc[:,i] = data.iloc[:,i].apply(lambda x: x.replace('.',''))

#Eliminar filas con IPv6, no se pueden representar como numeros para el qPCA
#Todos los IPv6s comienzan con 2001:dbb, comprobar si hay : entre las posiciones 3 y 5 del string
#para saber si es IPv6
indicesFilas=[]
for i in range (0,len(data)):
    if (data.iloc[i,0].find(':',3,5)!=-1):
        indicesFilas.append(i)

#Eliminar las filas con IPs detectadas como IPv6
data=data.drop(indicesFilas)

#Poner las columnas preprocesadas en tipo integer ()
data=data.astype('int64')

#Estandarizar datos 
data=StandardScaler().fit_transform(data)
#Pasar datos a dataframe 
data=pd.DataFrame(data)
#Reasignar los nombres a las columnas
data.columns = colNames

# Feature selection
kwargs = {'style': 1}
feats=SPEC.spec(data.to_numpy(), **kwargs)

#Indices de mayor a menor de los valores de ranking del SPEC feature selection
index = feats.argsort()[::-1]
fsData=data.iloc[:,index[0:4]]

#Correlaciones entre variables ordenadas de mayor a menor eliminando los pares centrales
correlations=data.corr(method='kendall').abs().unstack().sort_values(ascending=True).drop_duplicates().dropna()

#Obtener los nombres de los dos pares de variables menos correladas
feats=list(correlations.index[0]+correlations.index[1])
#Escoger las variables de la base de datos original
fsData=data[feats]

#Matriz de covarianza de los datos
cmat = np.cov(fsData, rowvar=False)
p=cmat/cmat.trace()

print(p)

In [ ]:
# Como no consigo descargar los datos originales, copio las mismas puertas que en circuito

In [5]:
# PROGRAMA PRINCIPAL

#Primer estado para la inicialización
stateVec = np.array([1/2, 1/2, 1/2, 1/2])

#Ciclo principal para actualizar los qubits data
for iter in range(6):
    #Crear circuito con 5 qubits
    circuit = QuantumCircuit(6,6)
    
    #Inicializacion qubits data
    circuit.initialize(stateVec,range(4,6))

    #Aplicar puertas de hadamard a qubits 0,1,2,3 para aplicar Quantum Amplitude Estimation sobre la matriz unitaria
    circuit.h([0,1,2,3])

    #Calcular rotaciones de la puerta de la matriz unitaria
    #(theta, phi, lamb) = qiskit.quantum_info.OneQubitEulerDecomposer(basis='U3').angles(expm(2*1j*np.pi*p))
    (theta, phi, lamb) = (0.0632, 4.63, -1.48)
    #Aplicar rotaciones
    circuit.cu(theta,phi,lamb,0, 3, [4,5])
    #Calcular rotaciones de la puerta de la matriz unitaria aplicada dos veces, Quantum Amplitude Estimation
    #(theta2, phi2, lamb2) = qiskit.quantum_info.OneQubitEulerDecomposer(basis='U3').angles(expm(2*1j*np.pi*p*2))
    (theta2, phi2, lamb2) = (0.13, -4.88, -1.4)
    #Aplicar rotaciones
    circuit.cu(theta2,phi2,lamb2,0, 2, [4,5])
    #Calcular rotaciones de la puerta de la matriz unitaria aplicada cuatro veces, Quantum Amplitude Estimation
    #(theta3, phi3, lamb3) = qiskit.quantum_info.OneQubitEulerDecomposer(basis='U3').angles(expm(2*1j*np.pi*p*4))
    (theta3, phi3, lamb3) = (0.292, 1.24, -1.24)
    #Aplicar rotaciones
    circuit.cu(theta3,phi3,lamb3,0, 1, [4,5])
    #Calcular rotaciones de la puerta de la matriz unitaria aplicada ocho veces, Quantum Amplitude Estimation
    #(theta4, phi4, lamb4) = qiskit.quantum_info.OneQubitEulerDecomposer(basis='U3').angles(expm(2*1j*np.pi*p*8))
    (theta4, phi4, lamb4) = (0.89, 1.01, -0.976)
    #Aplicar rotaciones
    circuit.cu(theta4,phi4,lamb4,0, 0, [4,5])

    #Aplicar Quantum Fourier Transform en los qubits 0,1,2,3
    circuit.h(0)
    circuit.crz(-np.pi/2,0,1)
    circuit.h(1)
    circuit.crz(-np.pi/2,1,2)
    circuit.crz(-np.pi/4,0,2)
    circuit.h(2)
    circuit.crz(-np.pi/2,2,3)
    circuit.crz(-np.pi/4,1,3)
    circuit.crz(-np.pi/8,0,3)
    circuit.h(3)

    #Medir los qubits
    circuit.measure([0,1,2,3,4,5], [0,1,2,3,4,5])

    #Seleccionar backend
    provider = IBMQ.get_provider(hub='ibm-q-lantik', group='ehu-upv', project='proyecto1')
    backend = provider.backend.ibmq_jakarta

    #Transpilar y quedarse con el circuito más corto
    best_depth = 10**10
    for i in range(50):
        trans_circuit = transpile(circuit, backend=backend, optimization_level=random.randint(0,3))
        aux_depth = trans_circuit.depth()
        if aux_depth < best_depth:
            best_circuit = trans_circuit
            best_depth = aux_depth

    print("Circuit depth = "+str(best_depth))  

    #Ejecutar circuito en el simulador y guardar resultados obtenidos
    #job = execute(best_circuit, backend=BasicAer.get_backend('qasm_simulator'), shots=10000)
    job = execute(best_circuit, backend=backend, shots=10000)
    
    #Obtener resultados, puede ocurrir que alguno de los estados de la base no se midan
    results = job.result().get_counts()    
    try:
        r1 = results['001111']
    except:
        r1 = 0
    try:
        r2 = results['011111']
    except:
        r2 = 0
    try:
        r3 = results['101111']
    except:
        r3 = 0
    try:
        r4 = results['111111']
    except:
        r4 = 0

    #Calcular coeficientes del eigenvector en proyeccion 1111 (00,01,10,11); segunda proyeccion en 1110 segundo eigenvec????? 
    denominator = r1 + r2 + r3 + r4
    alpha1 = np.sqrt(r1 / denominator)
    alpha2 = np.sqrt(r2 / denominator)
    alpha3 = np.sqrt(r3 / denominator)
    alpha4 = np.sqrt(r4 / denominator)

    #Guardar nuevo estado inicial para la siguiente ejecucion
    stateVec = [alpha1, alpha2, alpha3, alpha4]

    print("Eigenvector obtenido: ", stateVec)
    
    #Tiempo de espera para no sobrecargar de querys el servidor
    time.sleep(60)



Circuit depth = 89
Eigenvector obtenido:  [0.4246502900652006, 0.4616435357484827, 0.5121475197315839, 0.5867386940384682]
Circuit depth = 89
Eigenvector obtenido:  [0.492887262066433, 0.5420196589926082, 0.4317877695883728, 0.5261522196019802]
Circuit depth = 89
Eigenvector obtenido:  [0.49021431794612025, 0.46589082794780523, 0.5427463895880402, 0.4980582450917523]
Circuit depth = 89
Eigenvector obtenido:  [0.5094526324721704, 0.5168902906024488, 0.5456280085040742, 0.41901374569812083]
Circuit depth = 89
Eigenvector obtenido:  [0.48454371185234896, 0.5437390678561211, 0.41702882811414954, 0.5437390678561211]
Circuit depth = 89
Eigenvector obtenido:  [0.5030030300035687, 0.5264108981916501, 0.490880693673816, 0.4784513169075851]
